In [51]:
import boto3
import json

# Initialize Batch client
batch_client = boto3.client("batch", region_name="us-east-1")

# List of parameters for each job
# parameters = [{"param1": "boxscoreadvancedv3.BoxScoreAdvancedV3", "param2": "1"},
#               {"param1": "boxscoretraditionalv3.BoxScoreTraditionalV3", "param2": "2"}]

func1 = "BoxScoreAdvancedV3"
index1 = "1"

response = batch_client.submit_job(
    jobName=f"process-data-{func1}",
    jobQueue="fargate-job-queue",
    jobDefinition="fargate-job-definition:2",
    containerOverrides={
        "command": ["python", "games_process.py", func1, index1],
    }
)

print(f"Submitted job for {func1}: {response['jobId']}")


Submitted job for BoxScoreAdvancedV3: 5c672e82-da57-4005-bec3-2f9667412b54


In [31]:
import boto3

batch_client = boto3.client('batch', region_name='us-east-1')

response = batch_client.register_job_definition(
    jobDefinitionName='fargate-job-definition',
    type='container',
    containerProperties={
        'image': '585768178063.dkr.ecr.us-east-1.amazonaws.com/nbabigone:latest',  # Replace with your ECR image URL
        'command': ['python', 'games_process.py'],
        'executionRoleArn': 'arn:aws:iam::585768178063:role/FargateBatchExecutionRole',  # Replace for Fargate
        'jobRoleArn': 'arn:aws:iam::585768178063:role/FargateBatchExecutionRole',  # Replace with your job role ARN
        'resourceRequirements': [
            {
                'type': 'VCPU',
                'value': '1.0'
            },
            {
                'type': 'MEMORY',
                'value': '2048'
            }
        ],
        'networkConfiguration': {
            'assignPublicIp': 'ENABLED'  # For public subnets
        },
        'fargatePlatformConfiguration': {
            'platformVersion': 'LATEST'
        },
        'environment': [
            {
                'name': 'YOUR_ENV_VAR',
                'value': 'your_value'
            }
        ],
        'logConfiguration': {
            'logDriver': 'awslogs',
            'options': {
                'awslogs-group': '/aws/batch/job',
                'awslogs-region': 'us-east-1',
                'awslogs-stream-prefix': 'fargate-job'
            }
        }
    },
    platformCapabilities=[
        'FARGATE',
    ]
)

print(f"Job Definition ARN: {response['jobDefinitionArn']}")

Job Definition ARN: arn:aws:batch:us-east-1:585768178063:job-definition/fargate-job-definition:2


In [50]:
batch = boto3.client('batch', region_name='us-east-1')

response = batch.register_job_definition(
    jobDefinitionName='fargate-job-definition',
    type='container',
    containerProperties={
        'image': 'your-ecr-repository-url:latest',  # Replace with your ECR image URL
        'command': ['python', 'games_process.py'],
        'executionRoleArn': 'arn:aws:iam::585768178063:role/FargateBatchExecutionRole',
        'jobRoleArn': 'arn:aws:iam::585768178063:role/BatchTaskRole',  # Add the new task role
        'resourceRequirements': [
            {
                'type': 'VCPU',
                'value': '1.0'
            },
            {
                'type': 'MEMORY',
                'value': '2048'
            }
        ],
        'networkConfiguration': {
            'assignPublicIp': 'ENABLED'
        },
        'logConfiguration': {
            'logDriver': 'awslogs',
            'options': {
                'awslogs-group': '/aws/batch/job',
                'awslogs-region': 'us-east-1',
                'awslogs-stream-prefix': 'fargate-job'
            }
        }
    },
    platformCapabilities=[
        'FARGATE',
    ]
)